In [1]:
#Testing boto3 aws connection

import boto3

s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

crownfloats-test.com
elasticbeanstalk-us-east-1-862738521567
elasticbeanstalk-us-west-2-862738521567
hidden-6ix.com
loginfo-hidden6ix.com
redditpostestimator
www.hidden-6ix.com


In [2]:
###Starting with exercise 2, to provide observability and feedback on performance of task 1 

In [73]:
import datetime
import time
ec2_client = boto3.client('ec2') 
cloudwatch_client = boto3.client('cloudwatch')


In [138]:
#2a) See the status of all the diffferent servers being monitored
instance_status_paginator = ec2_client.get_paginator('describe_instance_status')
instance_status_iterator = instance_status_paginator.paginate()
instanceIds = []  ### Saved for termination
for instance_status in instance_status_iterator:
    for status in instance_status['InstanceStatuses']:
        instanceIds.append(status['InstanceId'])
        print(status['InstanceId'])
        print(status['InstanceState'])

i-02e5206bfe629e1c2
{'Code': 16, 'Name': 'running'}
i-039900f7f0d0d0373
{'Code': 16, 'Name': 'running'}


In [137]:
#2c) Get spot price for instances 

ec2 = boto3.resource('ec2')
instanceActualPrices = {} 
for instanceId in instanceIds: 
    instance = ec2.Instance(instanceId)
    if (instance.spot_instance_request_id != None):
        response = client.describe_spot_instance_requests(
            DryRun=False,
            SpotInstanceRequestIds=[
                instance.spot_instance_request_id,
            ]
        )
        instanceActualPrices[instanceId] = response['SpotInstanceRequests'][0]['SpotPrice']
print(instanceActualPrices)

{'i-039900f7f0d0d0373': '0.200000'}


In [33]:
#2a) See how many processes are running on it
# Methodology is to have hosts run important process that continually emits cloud watch metrics, which can be watched and alerted on


In [80]:
##############
# Script which emits cloudwatch metrics 
##############
while(True): 
    
    response = cloudwatch_client.put_metric_data(
        Namespace='chorus',
        MetricData=[
        {
            'MetricName': 'process1',  #Add more metrics for important proceses to observe
            'Dimensions': [
                {
                    'Name': 'isImportProcessRunning',
                    'Value': '1'
                },
            ],
            'StatisticValues': {
                   'SampleCount': 1,
                'Sum': 1,
                'Minimum': 1,
                'Maximum': 1
            }
        },
        ]
    )
    print('submitted')
    time.sleep(1)

submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted


KeyboardInterrupt: 

In [82]:
# Polling instance level cloudwatch metrics (it can take up to 15 minutes to see results)
response = cloudwatch_client.list_metrics(
    Namespace="chorus"
)
print(response)
for metric in response['Metrics']:
    print(metric)
    response = cloudwatch_client.get_metric_data(
        MetricDataQueries=[
            {
            'Id': 'chorusAITechnical',
            'MetricStat': {
                'Metric': metric,
                'Period': 300,
                'Stat': 'Sum',
                'Unit': 'None'
               },
            },
        ],
    StartTime=datetime.datetime(2015, 1, 1),
    EndTime=datetime.datetime(2019, 12, 12))
    print(response)
    print("======================================================")


{'Metrics': [{'Namespace': 'chorus', 'MetricName': 'importProcessRunning', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}, {'Namespace': 'chorus', 'MetricName': 'process1', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}], 'ResponseMetadata': {'RequestId': '8f50c9ef-72a4-11e9-a1db-0d43e0b61dc2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8f50c9ef-72a4-11e9-a1db-0d43e0b61dc2', 'content-type': 'text/xml', 'content-length': '834', 'date': 'Thu, 09 May 2019 21:51:10 GMT'}, 'RetryAttempts': 0}}
{'Namespace': 'chorus', 'MetricName': 'importProcessRunning', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}
{'MetricDataResults': [{'Id': 'chorusAITechnical', 'Label': 'importProcessRunning', 'Timestamps': [datetime.datetime(2019, 5, 9, 21, 40, tzinfo=tzutc()), datetime.datetime(2019, 5, 9, 21, 30, tzinfo=tzutc())], 'Values': [1.0, 200.0], 'StatusCode': 'PartialData'}], 'NextToken': 'ANUVeafcc1fwvCPLh4YhyJETEqmgu9sOdknDP2yucWQ+

In [146]:
#### 1 a) Monitor spot prices

def getSpotPrice(): 
    response = ec2_client.describe_spot_price_history(
        StartTime=datetime.datetime.today() - datetime.timedelta(days=1),
        InstanceTypes=['t3.micro'],MaxResults=1,ProductDescriptions=['Linux/UNIX (Amazon VPC)']
    )
    spot_price = response['SpotPriceHistory'][0]['SpotPrice']
    return spot_price





In [109]:
### 1b) Send message to machine to shutdown (building block)

def terminateSpotInstances(instanceIds): 
    response = ec2_client.terminate_instances(
        InstanceIds=instanceIds,
        DryRun=False #just to not accidentally cause problem
    )




ClientError: An error occurred (DryRunOperation) when calling the TerminateInstances operation: Request would have succeeded, but DryRun flag is set.

In [117]:
### 1e) Create a spot instance when time is ready




ready = True

def createNewSpotInstances(numberOfSpotInstances): 
    spot_price = "5.00"  ### Actually a very high value
    response = ec2_client.request_spot_instances(
        InstanceCount=1,
        LaunchSpecification={
            'ImageId': 'ami-061392db613a6357b',
            'InstanceType': 't1.micro' # I know it should be m4.xlarge, but to save money 
        },
        SpotPrice=spot_price
    )
    print(response)

    


{'SpotInstanceRequests': [{'CreateTime': datetime.datetime(2019, 5, 9, 22, 19, 54, tzinfo=tzutc()), 'LaunchSpecification': {'SecurityGroups': [{'GroupName': 'default', 'GroupId': 'sg-6eb7140a'}], 'ImageId': 'ami-061392db613a6357b', 'InstanceType': 't1.micro', 'Placement': {'AvailabilityZone': 'us-west-2d'}, 'SubnetId': 'subnet-3ffba717', 'Monitoring': {'Enabled': False}}, 'ProductDescription': 'Linux/UNIX', 'SpotInstanceRequestId': 'sir-x9ki9pqn', 'SpotPrice': '0.200000', 'State': 'open', 'Status': {'Code': 'pending-evaluation', 'Message': 'Your Spot request has been submitted for review, and is pending evaluation.', 'UpdateTime': datetime.datetime(2019, 5, 9, 22, 19, 54, tzinfo=tzutc())}, 'Type': 'one-time', 'InstanceInterruptionBehavior': 'terminate'}], 'ResponseMetadata': {'RequestId': '68a2c302-3449-4041-9fa8-ab9be0418265', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'content-length': '1646', 'date': 'Thu, 09 May 2019 22:19:53 GMT', 'server': 'A

In [148]:
#### Wrapping everything together

priceThreshold = 0.0002
numberOfInstancesToKeepRunning = 5
numberOfInstancesRunning = 0 
while(True): 
    currentSpotPrice = getSpotPrice()
    if(float(currentSpotPrice) > priceThreshold):
        instanceIdsToTerminate = instanceIdsToTerminate() ## TODO connect cloud watch monitoring of processes
        terminateSpotInstances(instanceIds)
        numberOfInstancesRunning = numberOfInstancesRunning - len(instanceIdsToTerminate)
    if(float(currentSpotPrice) < priceThreshold):
        createNewSpotInstances(numberOfInstancesToKeepRunning - numberOfInstancesRunning)
        
        
        
    
    



too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100
too high
0.003100


KeyboardInterrupt: 